# revere

In [1]:
import pandas as pd
import numpy as np
import pylab
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(color_codes=True)

In [205]:
df_customers = pd.read_pickle('customers_clean.pkl')
df_suppliers = pd.read_pickle('suppliers_clean.pkl')
df = pd.concat([df_customers, df_suppliers])
#df = df[df['start'] != df['end']]

In [206]:
df.head()

,id,start,end,rel_type,source_ticker,target_ticker,source_cusip,target_cusip
0,CUST-100000247,2016-11-03,2017-08-10,CUSTOMER,IW5-SG,ADRO-ID,Y7693G104,Y7087B109
1,CUST-100000500,2016-11-03,2017-08-10,CUSTOMER,IW5-SG,INTP-ID,Y7693G104,Y7127B135
2,CUST-100001093,2016-11-03,2017-08-10,CUSTOMER,OANDO-NG,XOM,V6796A108,30231G102
3,CUST-100001232,2016-11-03,2017-08-10,CUSTOMER,533278-IN,500470-IN,Y1668L107,Y8547N139
4,CUST-100001451,2016-11-03,2017-08-10,CUSTOMER,533278-IN,526371-IN,Y1668L107,Y6223W100


In [207]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227499 entries, 0 to 63235
Data columns (total 8 columns):
id               227499 non-null object
start            227499 non-null datetime64[ns]
end              227499 non-null datetime64[ns]
rel_type         227499 non-null object
source_ticker    227499 non-null object
target_ticker    227499 non-null object
source_cusip     227499 non-null object
target_cusip     227499 non-null object
dtypes: datetime64[ns](2), object(6)
memory usage: 15.6+ MB


In [208]:
source = df[['source_cusip','source_ticker']]
#source['cusip_8'] = source['source_cusip'].str[:8]
#source = source.rename(columns = {'cusip_8':'cusip'})
source = source.drop_duplicates(subset = 'source_cusip')
source = source.set_index('source_cusip')
source.head()

,source_ticker
source_cusip,
Y7693G104,IW5-SG
V6796A108,OANDO-NG
Y1668L107,533278-IN
J12459103,4924-JP
K07774126,BO-DK


In [69]:
source.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17100 entries, Y7693G10 to Q6630H10
Data columns (total 1 columns):
source_cusip    17100 non-null object
dtypes: object(1)
memory usage: 267.2+ KB


In [209]:
target = df[['target_cusip','target_ticker']]
# target['cusip'] = target['target_cusip'].str[:8]
#target = target.rename(columns = {'target_cusip':'cusip'})
target = target.drop_duplicates(subset = 'target_cusip')
target = target.set_index('target_cusip')
target.head()

,target_ticker
target_cusip,
Y7087B109,ADRO-ID
Y7127B135,INTP-ID
30231G102,XOM
Y8547N139,500470-IN
Y6223W100,526371-IN


In [71]:
target.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19050 entries, Y7087B10 to N3070G10
Data columns (total 1 columns):
target_cusip    19050 non-null object
dtypes: object(1)
memory usage: 297.7+ KB


In [210]:
cusip_all = pd.concat([source, target], axis =1, sort=False)
cusip_all = cusip_all.reset_index()
cusip_all = cusip_all.drop_duplicates(subset = 'index')

In [211]:
cusip_all = cusip_all.set_index('index')

In [212]:
len(cusip_all.index.unique())

23718

In [213]:
cusip_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23718 entries, Y7693G104 to N3070G108
Data columns (total 2 columns):
source_ticker    17128 non-null object
target_ticker    19088 non-null object
dtypes: object(2)
memory usage: 555.9+ KB


In [214]:
cusip_all.head()

,source_ticker,target_ticker
index,,
Y7693G104,IW5-SG,NaN
V6796A108,OANDO-NG,OANDO-NG
Y1668L107,533278-IN,533278-IN
J12459103,4924-JP,4924-JP
K07774126,BO-DK,BO-DK


# crsp

In [83]:
ncusip = pd.read_sas('ncusip.sas7bdat', encoding='iso-8859-1')
#ncusip['ncusip'] = sic['CUSIP']
#sic['ticker'] = sic['TICKER']

In [84]:
ncusip['ncusip'] = ncusip['NCUSIP']
ncusip['ticker'] = ncusip['TICKER']
ncusip = ncusip.drop_duplicates(subset='ncusip', keep='first')

In [85]:
ncusip = ncusip.set_index('ncusip')

In [86]:
ncusip.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18819 entries, 29274A10 to 88160R10
Data columns (total 7 columns):
PERMNO    18819 non-null float64
DATE      18819 non-null datetime64[ns]
CUSIP     18819 non-null object
NCUSIP    18818 non-null object
TICKER    18394 non-null object
SICCD     18810 non-null float64
ticker    18394 non-null object
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 1.1+ MB


In [87]:
ncusip.head()

,PERMNO,DATE,CUSIP,NCUSIP,TICKER,SICCD,ticker
ncusip,,,,,,,
29274A10,10001.0,2003-01-31,36720410,29274A10,EWST,4920.0,EWST
29274A20,10001.0,2008-02-29,36720410,29274A20,EWST,4920.0,EWST
29269V10,10001.0,2009-08-31,36720410,29269V10,EGAS,4920.0,EGAS
36720410,10001.0,2010-07-30,36720410,36720410,EGAS,4925.0,EGAS
05978R10,10002.0,2003-01-31,05978R10,05978R10,BTFG,6020.0,BTFG


# ncusip

In [88]:
df_ncusip = pd.concat([cusip_all, ncusip], axis =1, join='inner')

In [89]:
df_ncusip.head()

,source_cusip,target_cusip,PERMNO,DATE,CUSIP,NCUSIP,TICKER,SICCD,ticker
63040210,630402105,630402105,55993.0,2003-01-31,63040210,63040210,NSSC,3662.0,NSSC
71361F10,71361F100,71361F100,77868.0,2003-01-31,71361F10,71361F10,PRCP,3820.0,PRCP
03783310,037833100,037833100,14593.0,2003-01-31,03783310,03783310,AAPL,3573.0,AAPL
87891920,878919208,878919208,86787.0,2003-01-31,87891920,87891920,TEKC,7370.0,TEKC
87238210,872382106,NaN,74246.0,2003-01-31,87238210,87238210,TELOZ,6792.0,TELOZ


In [90]:
df_ncusip.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5137 entries, 63040210 to 75025X10
Data columns (total 9 columns):
source_cusip    4207 non-null object
target_cusip    4555 non-null object
PERMNO          5137 non-null float64
DATE            5137 non-null datetime64[ns]
CUSIP           5137 non-null object
NCUSIP          5137 non-null object
TICKER          5029 non-null object
SICCD           5136 non-null float64
ticker          5029 non-null object
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 401.3+ KB


# ticker

In [116]:
source_tickers = source.reset_index()

In [117]:
source_tickers = source_tickers.drop_duplicates(subset='source_ticker', keep='first')

In [118]:
source_tickers = source_tickers.set_index('source_ticker')

In [119]:
source_tickers.head()

,cusip,source_cusip
source_ticker,,
IW5-SG,Y7693G10,Y7693G104
OANDO-NG,V6796A10,V6796A108
533278-IN,Y1668L10,Y1668L107
4924-JP,J1245910,J12459103
BO-DK,K0777412,K07774126


In [121]:
target_tickers = target.reset_index()
target_tickers = target_tickers.drop_duplicates(subset='target_ticker', keep='first')
target_tickers = target_tickers.set_index('target_ticker')

In [122]:
target_tickers.head()

,cusip,target_cusip
target_ticker,,
ADRO-ID,Y7087B10,Y7087B109
INTP-ID,Y7127B13,Y7127B135
XOM,30231G10,30231G102
500470-IN,Y8547N13,Y8547N139
526371-IN,Y6223W10,Y6223W100


In [123]:
revere_ticker = pd.concat([source_tickers,target_tickers])

/Users/xingkong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [124]:
revere_ticker = revere_ticker.drop_duplicates(subset='cusip', keep='first')
revere_ticker.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23686 entries, IW5-SG to ESP-NL
Data columns (total 3 columns):
cusip           23686 non-null object
source_cusip    17100 non-null object
target_cusip    6586 non-null object
dtypes: object(3)
memory usage: 740.2+ KB


In [127]:
crsp_ticker = ncusip.reset_index()
crsp_ticker = crsp_ticker.drop_duplicates(subset='ticker', keep='first')
crsp_ticker = crsp_ticker.set_index('ticker')

In [128]:
#crsp_ticker = crsp_ticker.drop_duplicates(subset='CUSIP', keep='first')
crsp_ticker.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14585 entries, EWST to TSLA
Data columns (total 7 columns):
ncusip    14584 non-null object
PERMNO    14585 non-null float64
DATE      14585 non-null datetime64[ns]
CUSIP     14585 non-null object
NCUSIP    14584 non-null object
TICKER    14584 non-null object
SICCD     14578 non-null float64
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 911.6+ KB


In [129]:
df_ticker = pd.concat([revere_ticker, crsp_ticker], axis =1, join='inner')

In [130]:
df_ticker.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4349 entries, NSSC to RLGT
Data columns (total 10 columns):
cusip           4349 non-null object
source_cusip    3583 non-null object
target_cusip    766 non-null object
ncusip          4349 non-null object
PERMNO          4349 non-null float64
DATE            4349 non-null datetime64[ns]
CUSIP           4349 non-null object
NCUSIP          4349 non-null object
TICKER          4349 non-null object
SICCD           4349 non-null float64
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 373.7+ KB


In [131]:
ccm = pd.read_sas('ccmxpf_linktable.sas7bdat', encoding='iso-8859-1')
ccm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73930 entries, 0 to 73929
Data columns (total 9 columns):
gvkey        73930 non-null object
linkprim     73930 non-null object
liid         73930 non-null object
linktype     73930 non-null object
lpermno      32525 non-null float64
lpermco      32525 non-null float64
USEDFLAG     73930 non-null float64
linkdt       73930 non-null datetime64[ns]
linkenddt    63859 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(3), object(4)
memory usage: 5.1+ MB


In [134]:
ccm = ccm.drop_duplicates(subset='gvkey', keep='first')
ccm.head()

,gvkey,linkprim,liid,linktype,lpermno,lpermco,USEDFLAG,linkdt,linkenddt
0,001000,C,00X,NU,NaN,NaN,-1.0,1961-01-01,1970-09-29
3,001001,C,00X,NU,NaN,NaN,-1.0,1978-01-01,1983-09-19
5,001002,C,00X,NR,NaN,NaN,-1.0,1960-01-01,1970-09-29
9,001003,C,00X,NU,NaN,NaN,-1.0,1980-01-01,1983-12-06
12,001004,C,00X,NU,NaN,NaN,-1.0,1965-06-01,1968-04-29


In [135]:
ccm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27424 entries, 0 to 73927
Data columns (total 9 columns):
gvkey        27424 non-null object
linkprim     27424 non-null object
liid         27424 non-null object
linktype     27424 non-null object
lpermno      3904 non-null float64
lpermco      3904 non-null float64
USEDFLAG     27424 non-null float64
linkdt       27424 non-null datetime64[ns]
linkenddt    25966 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(3), object(4)
memory usage: 2.1+ MB


In [179]:
comp_cusip = pd.read_sas('comp_cusip.sas7bdat', encoding='iso-8859-1')

In [180]:
comp_cusip = comp_cusip.drop_duplicates(subset='cusip', keep='first')

In [181]:
comp_cusip = comp_cusip.set_index('tic')

In [182]:
len(comp_cusip.index.unique())

21604

In [183]:
comp_cusip.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21608 entries, AIR to GRIN
Data columns (total 12 columns):
gvkey       21608 non-null object
datadate    21608 non-null datetime64[ns]
fyear       21054 non-null float64
indfmt      21608 non-null object
consol      21608 non-null object
popsrc      21608 non-null object
datafmt     21608 non-null object
cusip       21607 non-null object
curcd       21054 non-null object
costat      21608 non-null object
gind        17137 non-null object
gsector     17137 non-null object
dtypes: datetime64[ns](1), float64(1), object(10)
memory usage: 2.1+ MB


In [184]:
comp_cusip.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,cusip,curcd,costat,gind,gsector
tic,,,,,,,,,,,,
AIR,001004,2003-05-31,2002.0,INDL,C,D,STD,000361105,USD,A,201010,20
4165A,001010,2003-12-31,2003.0,INDL,C,D,STD,00099V004,USD,I,203040,20
ADCT,001013,2003-10-31,2003.0,INDL,C,D,STD,000886309,USD,I,452010,45
IWKS,001021,2003-06-30,2003.0,INDL,C,D,STD,001058205,USD,I,351010,35
ALO.2,001034,2003-12-31,2003.0,INDL,C,D,STD,020813101,USD,I,352020,35


In [159]:
comp_revere = pd.concat([cusip_all, comp_cusip], axis =1, join='inner')

In [160]:
comp_revere.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5584 entries, 630402105 to 75025X100
Data columns (total 14 columns):
source_ticker    4363 non-null object
target_ticker    4809 non-null object
gvkey            5584 non-null object
datadate         5584 non-null datetime64[ns]
fyear            5431 non-null float64
indfmt           5584 non-null object
consol           5584 non-null object
popsrc           5584 non-null object
datafmt          5584 non-null object
tic              5584 non-null object
curcd            5431 non-null object
costat           5584 non-null object
gind             5568 non-null object
gsector          5568 non-null object
dtypes: datetime64[ns](1), float64(1), object(12)
memory usage: 654.4+ KB


In [185]:
merge_revere = cusip_all.reset_index()
merge_comp = comp_cusip.reset_index()

In [190]:
merge_comp['ticker'] = merge_comp['tic']
merge_comp.head()

,tic,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,cusip,curcd,costat,gind,gsector,ticker
0,AIR,001004,2003-05-31,2002.0,INDL,C,D,STD,000361105,USD,A,201010,20,AIR
1,4165A,001010,2003-12-31,2003.0,INDL,C,D,STD,00099V004,USD,I,203040,20,4165A
2,ADCT,001013,2003-10-31,2003.0,INDL,C,D,STD,000886309,USD,I,452010,45,ADCT
3,IWKS,001021,2003-06-30,2003.0,INDL,C,D,STD,001058205,USD,I,351010,35,IWKS
4,ALO.2,001034,2003-12-31,2003.0,INDL,C,D,STD,020813101,USD,I,352020,35,ALO.2


In [200]:
#revere_ticker = revere_ticker.reset_index()
revere_ticker['ticker'] = revere_ticker['index']
revere_ticker.head()

,level_0,index,cusip,source_cusip,target_cusip,ticker
0,0,IW5-SG,Y7693G10,Y7693G104,NaN,IW5-SG
1,1,OANDO-NG,V6796A10,V6796A108,NaN,OANDO-NG
2,2,533278-IN,Y1668L10,Y1668L107,NaN,533278-IN
3,3,4924-JP,J1245910,J12459103,NaN,4924-JP
4,4,BO-DK,K0777412,K07774126,NaN,BO-DK


In [192]:
merge_comp.head()

,tic,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,cusip,curcd,costat,gind,gsector,ticker
0,AIR,001004,2003-05-31,2002.0,INDL,C,D,STD,000361105,USD,A,201010,20,AIR
1,4165A,001010,2003-12-31,2003.0,INDL,C,D,STD,00099V004,USD,I,203040,20,4165A
2,ADCT,001013,2003-10-31,2003.0,INDL,C,D,STD,000886309,USD,I,452010,45,ADCT
3,IWKS,001021,2003-06-30,2003.0,INDL,C,D,STD,001058205,USD,I,351010,35,IWKS
4,ALO.2,001034,2003-12-31,2003.0,INDL,C,D,STD,020813101,USD,I,352020,35,ALO.2


In [202]:
revere_ticker.head()

,level_0,index,cusip,source_cusip,target_cusip,ticker
0,0,IW5-SG,Y7693G10,Y7693G104,NaN,IW5-SG
1,1,OANDO-NG,V6796A10,V6796A108,NaN,OANDO-NG
2,2,533278-IN,Y1668L10,Y1668L107,NaN,533278-IN
3,3,4924-JP,J1245910,J12459103,NaN,4924-JP
4,4,BO-DK,K0777412,K07774126,NaN,BO-DK


In [203]:
revere_comp = pd.merge(revere_ticker,merge_comp, on = 'ticker')

In [204]:
revere_comp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4877 entries, 0 to 4876
Data columns (total 19 columns):
level_0         4877 non-null int64
index           4877 non-null object
cusip_x         4877 non-null object
source_cusip    3902 non-null object
target_cusip    975 non-null object
ticker          4877 non-null object
tic             4877 non-null object
gvkey           4877 non-null object
datadate        4877 non-null datetime64[ns]
fyear           4739 non-null float64
indfmt          4877 non-null object
consol          4877 non-null object
popsrc          4877 non-null object
datafmt         4877 non-null object
cusip_y         4877 non-null object
curcd           4739 non-null object
costat          4877 non-null object
gind            4855 non-null object
gsector         4855 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(16)
memory usage: 762.0+ KB
